# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [71]:
# Write your code below.
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [72]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [73]:
import os
from glob import glob
PRICE_DATA = os.getenv('PRICE_DATA')
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [74]:
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/dask/dataframe/core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


In [75]:
dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign( 
                        Close_lag_1 = x['Close'].shift(1),
                        Close_lag_adj = x['Adj Close'].shift(1)
))

print(dd_shift)

dd_feat = dd_shift.assign(
    returns =  lambda x: x['Close']/x['Close_lag_1'] - 1,
    hi_lo_range =  lambda x: x['High'] - x['Low']
) 

print(dd_feat)

Dask DataFrame Structure:
                                  Date Adj Close    Close     High      Low     Open   Volume   Year Close_lag_1 Close_lag_adj
npartitions=13078                                                                                                             
                   datetime64[ns, UTC]   float64  float64  float64  float64  float64  float64  int32     float64       float64
                                   ...       ...      ...      ...      ...      ...      ...    ...         ...           ...
...                                ...       ...      ...      ...      ...      ...      ...    ...         ...           ...
                                   ...       ...      ...      ...      ...      ...      ...    ...         ...           ...
                                   ...       ...      ...      ...      ...      ...      ...    ...         ...           ...
Dask Name: lambda, 9 graph layers
Dask DataFrame Structure:
                         

/var/folders/_n/w8k9zwj57xg4g7tchwrj9h1c0000gn/T/ipykernel_58310/2682938939.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [76]:
# Write your code below.
pd_df_feat = dd_feat.compute()


In [105]:
df = pd_df_feat.copy()
df['mv_avg'] = df['returns'].rolling(window=10).mean()
df


Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Close_lag_adj,returns,hi_lo_range,mv_avg
Ticker,,,,,,,,,,,,,
DOV,2005-01-03 00:00:00+00:00,18.734230,27.604422,28.227835,27.544090,28.107174,762305.0,2005,NaN,NaN,NaN,0.683744,NaN
DOV,2005-01-04 00:00:00+00:00,18.647795,27.477058,28.013327,27.403320,27.865852,1330378.0,2005,27.604422,18.734230,-0.004614,0.610006,NaN
DOV,2005-01-05 00:00:00+00:00,18.283842,26.940788,27.785412,26.907272,27.477058,1176575.0,2005,27.477058,18.647795,-0.019517,0.878139,NaN
DOV,2005-01-06 00:00:00+00:00,18.110966,26.686062,26.981009,26.625732,26.981009,1395122.0,2005,26.940788,18.283842,-0.009455,0.355276,NaN
DOV,2005-01-07 00:00:00+00:00,17.847099,26.297268,26.726282,26.277157,26.625732,1192984.0,2005,26.686062,18.110966,-0.014569,0.449125,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
CTLT,2010-12-27 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2010,NaN,NaN,NaN,NaN,NaN
CTLT,2010-12-28 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2010,NaN,NaN,NaN,NaN,NaN
CTLT,2010-12-29 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2010,NaN,NaN,NaN,NaN,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.